In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\david\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\david\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\david\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\david\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv("owid-co2-data.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.isnull().sum().sum()

In [ ]:
sns.heatmap(df.isnull())
plt.show()

In [ ]:
df.info()

In [ ]:
corr = df.select_dtypes(include=['int64','float64']).corr()

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix of Process Variables")
plt.show()

In [ ]:
missing_frac = df.isna().mean().sort_values(ascending=False)
missing_frac.head(79)

In [ ]:
df.select_dtypes(include="number").corr()['co2_per_capita'].sort_values(ascending=False)

In [ ]:
target = "co2_per_capita"

features = [
    "energy_per_capita",
    "primary_energy_consumption",
    "coal_co2",
    "oil_co2",
    "gas_co2",
    "cement_co2",
    "flaring_co2",
    "gdp",
    "population"
]

In [ ]:
id_cols = ["country", "year"]

final_cols = id_cols + features + [target]

df_model = df[final_cols].copy()

In [ ]:
df_model.info()

In [ ]:
df_model.isna().mean().sort_values(ascending=False)

In [ ]:
core_required = ["co2_per_capita", "energy_per_capita", "population"]
df_model = df_model.dropna(subset=core_required)

In [ ]:
df_model.shape

In [ ]:
df_model.isna().sum()

In [ ]:
for col in ["coal_co2", "oil_co2", "gas_co2", "cement_co2"]:
    df_model[col] = (
        df_model
        .groupby("country")[col]
        .transform(lambda x: x.fillna(x.median()))
    )

In [ ]:
df_model["gdp"] = (
    df_model
    .sort_values("year")
    .groupby("country")["gdp"]
    .transform(lambda x: x.interpolate(method="linear"))
)

df_model["gdp"] = (
    df_model
    .groupby("country")["gdp"]
    .transform(lambda x: x.fillna(x.median()))
)

In [ ]:
import numpy as np

def impute_flaring(group):
    if (group > 0).any():
        return group.fillna(group.median())
    else:
        return group.fillna(0)

df_model["flaring_co2"] = (
    df_model
    .groupby("country")["flaring_co2"]
    .transform(impute_flaring)
)

In [ ]:
df_model.isna().sum()

In [ ]:
df_final = df_model.dropna(subset=final_cols)

In [ ]:
df_final.isna().sum()

In [ ]:
df_final.shape

In [ ]:
num_cols = [
    "energy_per_capita",
    "primary_energy_consumption",
    "coal_co2",
    "oil_co2",
    "gas_co2",
    "cement_co2",
    "flaring_co2",
    "gdp",
    "population",
    "co2_per_capita"
]

for col in num_cols:
    sns.boxplot(x=df_final[col])
    plt.title(col)
    plt.show()


In [ ]:
for i in df_final.select_dtypes(include="float64").columns:
    sns.distplot(df_final[i])
    plt.show()

In [ ]:
for col in log_cols:
    df_final[col] = np.log1p(df_final[col])


In [ ]:
for col in num_cols:
    sns.boxplot(x=df_final[col])
    plt.title(col)
    plt.show() 

In [ ]:
corr = df_final.select_dtypes(include=['float64']).corr()

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix of Process Variables")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X = df_final[features]
y = df_final["co2_per_capita"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

y_pred_lr = lr.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

rmse = mean_squared_error(y_test, y_pred_lr, squared=False)
r2 = r2_score(y_test, y_pred_lr)

rmse, r2

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_pred_rf = rf.predict(X_test)

rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
r2_rf = r2_score(y_test, y_pred_rf)

rmse_rf, r2_rf

In [ ]:
feature_importance = pd.Series(
    rf.feature_importances_,
    index=features
).sort_values(ascending=False)

feature_importance

In [ ]:
feature_importance.plot(kind="bar", figsize=(10,5))
plt.ylabel("Importance")
plt.title("Random Forest Feature Importance")
plt.show()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

In [ ]:
gbr.fit(X_train, y_train)

In [ ]:
y_pred_gbr = gbr.predict(X_test)

rmse_gbr = mean_squared_error(y_test, y_pred_gbr, squared=False)
r2_gbr = r2_score(y_test, y_pred_gbr)

rmse_gbr, r2_gbr

In [ ]:
gb_importance = pd.Series(
    gbr.feature_importances_,
    index=features
).sort_values(ascending=False)

gb_importance

In [ ]:
gb_importance.plot(kind="bar", figsize=(10,5))
plt.title("Gradient Boosting Feature Importance")
plt.ylabel("Importance")
plt.show()

In [ ]:
results = pd.DataFrame({
    "Model": ["Linear Regression", "Random Forest", "Gradient Boosting"],
    "RMSE": [rmse, rmse_rf, rmse_gbr],
    "R2": [r2, r2_rf, r2_gbr]
})

results

In [ ]:
models = {
    "Linear Regression": lr,
    "Random Forest": rf,
    "Gradient Boosting": gbr
}

predictions = {
    name: model.predict(X_test)
    for name, model in models.items()
}

In [ ]:
def plot_predicted_vs_actual(y_true, y_pred, model_name):
    plt.figure(figsize=(6, 6))
    
    plt.scatter(
        y_true, 
        y_pred, 
        alpha=0.4
    )
    
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    plt.plot(
        [min_val, max_val], 
        [min_val, max_val], 
        linestyle="--"
    )
    
    plt.xlabel("Actual log(CO₂ per capita)")
    plt.ylabel("Predicted log(CO₂ per capita)")
    plt.title(f"Predicted vs Actual — {model_name}")
    
    plt.tight_layout()
    plt.show()

In [ ]:
for name, y_pred in predictions.items():
    plot_predicted_vs_actual(y_test, y_pred, name)